In [ ]:
import torch #model
import torch.nn as nn #neural network
from torch.utils.data import Dataset #datasets
import pandas as pd #read csv and data processing
import matplotlib.pyplot as plt #plotting and visualization

In [ ]:
#add headers and read dataset
df = pd.read_csv("/content/drive/MyDrive/Datasets/lichess_puzzle.csv", usecols=[1], header=None)

df.head()

,1
0,r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - ...
1,5rk1/1p3ppp/pq3b2/8/8/1P1Q1N2/P4PPP/3R2K1 w - ...
2,r2qr1k1/b1p2ppp/pp4n1/P1P1p3/4P1n1/B2P2Pb/3NBP...
3,r4rk1/pp3ppp/2n1b3/q1pp2B1/8/P1Q2NP1/1PP1PP1P/...
4,3q1rk1/1pp3pp/5p1P/4pPP1/rb1pP3/3P1N2/b1P1B3/2...


In [ ]:
#empty is index 0
#w pawn is 1
#w knight is 2
#w bishop is 3
#w rook is 4
#w queen is 5
#w king is 6
#b pawn is 7
#b knight is 8
#b bishop is 9
#b rook is 10
#b queen is 11
#b king is 12

x_train = torch.zeros(1000, 64)
y_train = torch.zeros(1000, 832)

x_test = torch.zeros(200, 64)
y_test = torch.zeros(200, 832)

#trim dataset to top 1000
df_train = df[:1000]
df_test = df[1000:1200]


#going through dataset
#worst fen parser ever created but regex is hard
def ParseFen(data, x, y):
  for i in range(len(data)):
    #throw away everything except position of the pieces
    position = data.iloc[i, 0].split(' ')[0]
  
    xindex = 0
    yindex = 0
  
    for char in position:
      if char.isdigit():
        for j in range(int(char)):
          x[i, xindex] = 0
          xindex += 1

          y[i, yindex] = 1
          yindex += 13
    
      if char == 'P':
        x[i, xindex] = 1
        xindex += 1

        y[i, yindex + 1] = 1
        yindex += 13

      if char == 'N':
        x[i, xindex] = 1
        xindex += 1

        y[i, yindex + 2] = 1
        yindex += 13

      if char == 'B':
        x[i, xindex] = 1
        xindex += 1

        y[i, yindex + 3] = 1
        yindex += 13

      if char == 'R':
        x[i, xindex] = 1
        xindex += 1

        y[i, yindex + 4] = 1
        yindex += 13
      
      if char == 'Q':
        x[i, xindex] = 1
        xindex += 1

        y[i, yindex + 5] = 1
        yindex += 13

      if char == 'K':
        x[i, xindex] = 1
        xindex += 1

        y[i, yindex + 6] = 1
        yindex += 13

      if char == 'p':
        x[i, xindex] = 1
        xindex += 1

        y[i, yindex + 7] = 1
        yindex += 13

      if char == 'n':
        x[i, xindex] = 1
        xindex += 1

        y[i, yindex + 8] = 1
        yindex += 13

      if char == 'b':
        x[i, xindex] = 1
        xindex += 1

        y[i, yindex + 9] = 1
        yindex += 13

      if char == 'r':
        x[i, xindex] = 1
        xindex += 1

        y[i, yindex + 10] = 1
        yindex += 13
      
      if char == 'q':
        x[i, xindex] = 1
        xindex += 1

        y[i, yindex + 11] = 1
        yindex += 13

      if char == 'k':
        x[i, xindex] = 1
        xindex += 1

        y[i, yindex + 12] = 1
        yindex += 13   

ParseFen(df_train, x_train, y_train)
ParseFen(df_test, x_test, y_test)

print(df_train.iloc[0])
print(x_train[0])

1    r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - ...
Name: 0, dtype: object
tensor([1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0.,
        0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1.,
        1., 1., 0., 0., 0., 0., 0., 0., 0., 1.])


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(64, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 832)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=64, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=832, bias=True)
  )
)


In [ ]:
#training
alpha = 0.001

#loss and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=alpha)
loss_fn = torch.nn.BCEWithLogitsLoss()  

n_total_steps = len(x_train)
n_epochs = 20

for epoch in range(n_epochs):
  for i in range(len(x_train)):
    #forward
    output = model(x_train[i])
    loss = 0
    #cross entropy for each square
    for j in range(64):
      loss += loss_fn(output[j * 13:(j + 1) * 13], y_train[i, j * 13:(j + 1) * 13])
    #backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if(i+1)%10==0:
      print(f'epoch {epoch + 1} / {n_epochs}, step {i + 1} / {n_total_steps}, loss = {loss.item():.4f}')


epoch 1 / 20, step 10 / 1000, loss = 21.0367
epoch 1 / 20, step 20 / 1000, loss = 9.1545
epoch 1 / 20, step 30 / 1000, loss = 9.0081
epoch 1 / 20, step 40 / 1000, loss = 10.7541
epoch 1 / 20, step 50 / 1000, loss = 7.9300
epoch 1 / 20, step 60 / 1000, loss = 10.3542
epoch 1 / 20, step 70 / 1000, loss = 8.9530
epoch 1 / 20, step 80 / 1000, loss = 7.6225
epoch 1 / 20, step 90 / 1000, loss = 8.8271
epoch 1 / 20, step 100 / 1000, loss = 7.9474
epoch 1 / 20, step 110 / 1000, loss = 9.0145
epoch 1 / 20, step 120 / 1000, loss = 8.4201
epoch 1 / 20, step 130 / 1000, loss = 6.9656
epoch 1 / 20, step 140 / 1000, loss = 6.4585
epoch 1 / 20, step 150 / 1000, loss = 7.8153
epoch 1 / 20, step 160 / 1000, loss = 7.5490
epoch 1 / 20, step 170 / 1000, loss = 9.7613
epoch 1 / 20, step 180 / 1000, loss = 8.4194
epoch 1 / 20, step 190 / 1000, loss = 7.3488
epoch 1 / 20, step 200 / 1000, loss = 8.8143
epoch 1 / 20, step 210 / 1000, loss = 9.0585
epoch 1 / 20, step 220 / 1000, loss = 8.8428
epoch 1 / 20, st

In [ ]:
#test
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  
  for i in range(len(x_test)):
    #trained prediction
    output = model(x_test[i])
    
    #cross value, index
    for j in range(64):
      prediction = torch.argmax(output[j * 13:(j + 1) * 13])
      test = torch.argmax(y_test[i, j * 13:(j + 1) * 13])
      if prediction == test:
        n_correct += 1

      n_samples += 1

#this is accuracy of squares not of entire board
acc = 100.0 * n_correct / n_samples
print(f'accuracy = {acc}%')

accuracy = 87.28125%


In [ ]:
#starting position
test_input = torch.zeros(64)
for i in range(8):
  test_input[i] = 1;
  test_input[63 - i] = 1;

#get prediction
hypothesis = model(test_input)

#display
piecesdict = {
  0: " ",
  1: "P",
  2: "N",
  3: "B",
  4: "R",
  5: "Q",
  6: "K",
  7: "p",
  8: "n",
  9: "b",
  10: "r",
  11: "q",
  12: "k",
}

for i in range(8):
  for j in range(8):

    highest = 0
    highest_index = 0
    for k in range(13):
      if hypothesis[(i * 8 + j) * 13 + k] > highest:
        highest = hypothesis[(i * 8 + j) * 13 + k]
        highest_index = k
    if j < 7:
      print(piecesdict[highest_index], end =" ")
    else:       
      print(piecesdict[highest_index])

r n b q k b n r
p p           p
    n          
               
               
               
          P   P
R N B Q K B   R
